# Data Wrangling SQL

 To get the following results, you should run the 'Data Wrangling SQL.ipynb'

## OpenStreetMap

OpenStreetMap (OSM) foundation is building free and editable map of the world, enabling the development of freely-reusable geospatial data. The data from OpenStreetMap is being used by many applications such as GoogleMaps, Foursquare and Craigslist. To look at the map, or download your area of interest, you can visit http://www.openstreetmap.org website. 


## Area Chosen

For this project, I chose Chicago, the Windy City in the US. This is the city where I got my undergrad degree.

The original file is about 2 GB in size; I use a sample file about 50MB to perform my initial analysis on. Finally, I run it on the original file to create the CSV files for my database. 


## Data Exploration

Using ET.iterparse (i.e. iterative parsing) is efficient here since the original file is too large for processing the whole thing.

The main problem we encountered in the dataset is the street name inconsistencies. Below is the old name corrected with the better name. 
- Avenue (starting with capital letter)
- Ave
- Ave.
- avenue (starting with small letter)

To be able to process the data, we need to make these street types uniform. In case we are later searching for specific Avenue names, we can do a quick search on all street types that have the word 'Avenue' in them and we can make sure that we are not missing anything with abrreviations of Avenue.

## Auditing Postcodes

Postcodes are another inconsistent type of data that is entered into the map. The inconsistency is either in how they are represented (with the city abbreviation or without) or how long they are.


The 'get_postcode' function will take the 'tag' element as an input and return the elements for which the keys are equal to 'addr:postcode' 

The 'audit' function, like the one for street names, parses the XML file and iterates through node and way elements. It extracts the value attribute (i.e. the postcode) and add it to the 'dicti' dictionary.

### Different Postcodes and Ways to Clean Them up
The output shows that the postcodes are in these formats:
- A 5-digit format (e.g. 12345)
- A 5-digit format followed by more numbers after a hyphen (e.g. 12345-6789)

To deal with the postcodes, I divide them into different categories:
- First category include the ones:
    - Where the length equals to 5 (e.g. 12345)
    - Where the length is longer than 5, and they contain characters (like abbreviations of a city) 
- Second category include the ones:
    - Where the length is longer than 5, and they are followed by a hyphen (e.g. 12345-6789)
    
- Third category include the ones:
    - Where the length is longer than 5, but are not followed by any hyphen (e.g. 123456)
    - Where the length is shorter than 5 (e.g. 1234, 515)
    - Where the postcode equals to 'IL'
 

## Preparing the Data for the Database

To load the data to the SQLite database, I need to transfer it from the XML file to CSV files. I create multiple CSV files, and later create the corresponding tables in my database based on them.

The CSV files I want to have are:
- Node
- Node_tags
- Way
- Way_tags
- Way_nodes

Each of these CSV files contains different columns and stores data based on those columns. The columns used in the CSV files will be the table columns in the database. This is the schema:
- NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
- NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
- WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
- WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
- WAY_NODES_FIELDS = ['id', 'node_id', 'position']



## Data Overview

I want to get some information regarding the CSV files and the database I created.By importing 'hurry.filesize' I can translate the file sizes from bytes to KB or MB. 

In [8]:
from pprint import pprint
import os
from hurry.filesize import size 
dirpath = 'submission'


files_list = []
for path, dirs, files in os.walk(dirpath):
    files_list.extend([(filename, size(os.path.getsize(os.path.join(path, filename)))) for filename in files])

for filename, size in files_list:
    print '{:.<40s}: {:5s}'.format(filename,size)

Now that I have audited and cleaned the data and transfered everything into table in my database, I can start running queries on it. The queries answer many questions such as:   
- Number of nodes
- Number of way
- Number of unique users
- Most contributing users
- Number of users who contributed only once
- Top 10 amenitie
- Shops 
- Users who added amenities 


In [9]:
import sqlite3

sqlite_file = 'db.sqlite'
con = sqlite3.connect(sqlite_file)
cur = con.cursor()

### Number of nodes

In [10]:
def number_of_nodes():
    output = cur.execute('SELECT COUNT(*) FROM nodes')
    return output.fetchone()[0]

print 'Number of nodes: \n' , number_of_nodes()

Number of nodes: 
8701756


### Number of ways

In [11]:
def number_of_ways():
    output = cur.execute('SELECT COUNT(*) FROM ways')
    return output.fetchone()[0]

print 'Number of ways: \n' , number_of_ways()

Number of ways: 
1231106


### Number of unique users

In [12]:
def number_of_unique_users():
    output = cur.execute('SELECT COUNT(DISTINCT e.uid) FROM \
                         (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e')
    return output.fetchone()[0]

print 'Number of unique users: \n' , number_of_unique_users()

Number of unique users: 
2823


### Most contributing users

In [13]:
def most_contributing_users():
    
    output = cur.execute('SELECT e.user, COUNT(*) as num FROM \
                         (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e \
                         GROUP BY e.user \
                         ORDER BY num DESC \
                         LIMIT 10 ')
    pprint(output.fetchall())
    return output.fetchall()

print 'Most contributing users: \n'
most_contributing_users()

Most contributing users: 

[(u'chicago-buildings', 5605968),
 (u'Umbugbene', 1091115),
 (u'woodpeck_fixbot', 219369),
 (u'alexrudd (NHD)', 204341),
 (u'g246020', 107386),
 (u'patester24', 105214),
 (u'mpinnau', 103495),
 (u'asdf1234', 101397),
 (u'Oak_Park_IL', 101251),
 (u'TIGERcnl', 93141)]


[]

### Number of users who contributed once

In [14]:
def number_of_users_contributed_once():
    
    output = cur.execute('SELECT COUNT(*) FROM \
                             (SELECT e.user, COUNT(*) as num FROM \
                                 (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e \
                                  GROUP BY e.user \
                                  HAVING num = 1) u')
    
    return output.fetchone()[0]
                         
print 'Number of users who have contributed once: \n', number_of_users_contributed_once()

Number of users who have contributed once: 
636


### Top 10 amenities 

In [15]:
def top_ten_amenities_in_sf():
    output = cur.execute('SELECT value, COUNT(*) as num FROM nodes_tags\
                            WHERE key="amenity" \
                            GROUP BY value \
                            ORDER BY num DESC \
                            LIMIT 20' )
    pprint(output.fetchall())
    return output.fetchall()

print 'Top ten amenities: \n'
top_ten_amenities_in_sf()

Top ten amenities: 

[(u'place_of_worship', 3038),
 (u'school', 1906),
 (u'restaurant', 1568),
 (u'fast_food', 899),
 (u'parking', 603),
 (u'cafe', 450),
 (u'bench', 426),
 (u'bicycle_parking', 414),
 (u'fuel', 388),
 (u'bicycle_rental', 361),
 (u'bank', 330),
 (u'drinking_water', 282),
 (u'bar', 257),
 (u'fountain', 229),
 (u'grave_yard', 216),
 (u'shelter', 204),
 (u'fire_station', 192),
 (u'toilets', 179),
 (u'pharmacy', 166),
 (u'pub', 162)]


[]

### Top 10 cuisines

In [16]:
def cuisines_in_sf():
    output = cur.execute ('SELECT value, COUNT(*) as num FROM ways_tags \
                           WHERE key="cuisine" \
                           GROUP BY value \
                           ORDER BY num DESC \
                           LIMIT 10')
    pprint(output.fetchall())
    return output.fetchall()

print 'Top 10 cuisines: \n'
cuisines_in_sf()

Top 10 cuisines: 

[(u'burger', 340),
 (u'mexican', 72),
 (u'chicken', 64),
 (u'pizza', 58),
 (u'american', 53),
 (u'coffee_shop', 36),
 (u'sandwich', 31),
 (u'italian', 30),
 (u'chinese', 17),
 (u'ice_cream', 16)]


[]

### Different types of shops

### Users who added amenities to the map



In [17]:
def shops_in_sf():
    output = cur.execute('SELECT value, COUNT(*) as num FROM nodes_tags\
                            WHERE key="shop" \
                            GROUP BY value \
                            ORDER BY num DESC' )
    pprint.pprint(output.fetchall())
    return output.fetchall()

print 'Different types of shops: \n'
top_ten_amenities_in_sf()

Different types of shops: 

[(u'place_of_worship', 3038),
 (u'school', 1906),
 (u'restaurant', 1568),
 (u'fast_food', 899),
 (u'parking', 603),
 (u'cafe', 450),
 (u'bench', 426),
 (u'bicycle_parking', 414),
 (u'fuel', 388),
 (u'bicycle_rental', 361),
 (u'bank', 330),
 (u'drinking_water', 282),
 (u'bar', 257),
 (u'fountain', 229),
 (u'grave_yard', 216),
 (u'shelter', 204),
 (u'fire_station', 192),
 (u'toilets', 179),
 (u'pharmacy', 166),
 (u'pub', 162)]


[]

In [18]:
def users_who_added_amenity():
    output = cur.execute('SELECT DISTINCT(nodes.user), nodes_tags.value FROM \
                            nodes join nodes_tags \
                            on nodes.id=nodes_tags.id \
                            WHERE key="amenity" \
                            GROUP BY value \
                            LIMIT 10' ) 
    pprint(output.fetchall())
    return output.fetchall()

print 'Users who added amenity to the map: \n'
users_who_added_amenity()

Users who added amenity to the map: 

[(u'DACGroup', u'Family Restaurant'),
 (u'DACGroup', u'Furniture Store'),
 (u'Dark Asteroid', u'Lombard Village Hall'),
 (u'DACGroup',
  u'Portable Toilet Supplier;Trailer Rental Service;Construction Equipment Supplier;Fence Contractor'),
 (u'FrankRKryzak', u'arts_centre'),
 (u'Zol87', u'artwork'),
 (u'Tomasz11', u'atm'),
 (u'PhQ', u'baby_hatch'),
 (u'Tomasz11', u'bank'),
 (u'Umbugbene', u'banquet_hall')]


[]

### List of postcodes


In [19]:
def list_of_postcodes():
    output = cur.execute('SELECT e.value, COUNT(*) as num FROM \
                            (SELECT value FROM nodes_tags WHERE key="postcode"\
                             UNION ALL SELECT value FROM ways_tags WHERE key="postcode") e \
                            GROUP BY e.value \
                            ORDER BY num DESC \
                            LIMIT 5' ) 
    pprint(output.fetchall())
    return output.fetchall()

print 'List of postcodes: \n'
list_of_postcodes()


List of postcodes: 

[(u'60201', 9392),
 (u'60202', 7727),
 (u'60305', 1720),
 (u'60564', 1684),
 (u'60136', 1306)]


[]

### Amenities 

we checked to see what the amenities around this area are. Since the list was quite long, I limited it to the first 20 amenities with the highest number.

In [20]:
def amenities_around_47906():
    output = cur.execute('SELECT nodes_tags.value, COUNT(*) as num \
                          FROM nodes_tags \
                            JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE key="amenity") AS amenities \
                            ON nodes_tags.id = amenities.id \
                            WHERE nodes_tags.key="amenity"\
                            GROUP BY nodes_tags.value \
                            ORDER BY num DESC \
                            LIMIT 20' ) 
    pprint(output.fetchall())
    return output.fetchall()

print 'Amenities around  \n'
amenities_around_47906()

Amenities around  

[(u'place_of_worship', 3038),
 (u'school', 1906),
 (u'restaurant', 1568),
 (u'fast_food', 899),
 (u'parking', 603),
 (u'cafe', 450),
 (u'bench', 426),
 (u'bicycle_parking', 414),
 (u'fuel', 388),
 (u'bicycle_rental', 361),
 (u'bank', 330),
 (u'drinking_water', 282),
 (u'bar', 257),
 (u'fountain', 229),
 (u'grave_yard', 216),
 (u'shelter', 204),
 (u'fire_station', 192),
 (u'toilets', 179),
 (u'pharmacy', 166),
 (u'pub', 162)]


[]

### Popular Cafes

In [22]:
def most_popular_cafes():
    output = cur.execute('SELECT nodes_tags.value, COUNT(*) as num \
                          FROM nodes_tags \
                            JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value="coffee_shop") AS cafes \
                            ON nodes_tags.id = cafes.id \
                            WHERE nodes_tags.key="name"\
                            GROUP BY nodes_tags.value \
                            ORDER BY num DESC \
                            LIMIT 10' ) 
    pprint(output.fetchall())
    return output.fetchall()

print 'Most popular cafes: \n'
most_popular_cafes()

Most popular cafes: 

[(u'Starbucks', 50),
 (u"Dunkin' Donuts", 13),
 (u'Starbucks Coffee', 10),
 (u"Peet's Coffee & Tea", 3),
 (u'Intelligentsia', 2),
 (u'Intelligentsia Coffee', 2),
 (u'Blue Max Coffee', 1),
 (u'Bow Truss Coffee Roasters', 1),
 (u'Brew Brew Coffee Lounge', 1),
 (u'Bridgeport Coffeehouse', 1)]


[]

# Reference

https://github.com/Dalaska/Udacity-Data-Wrangling-Clean-OpenStreetMap

https://github.com/bestkao/data-wrangling-with-openstreetmap-and-mongodb

https://discussions.udacity.com/t/display-files-and-their-sizes-in-directory/186741

https://github.com/Nazaniiin/DataWrangling_OpenStreetMap

https://github.com/alphagammamle/Udacity-Data-Analyst-Nanodegree/tree/master/P3-OpenStreetMap-Wrangling-with-SQL

https://github.com/alphagammamle/OpenStreetMap-Toronto

https://github.com/paul-reiners/udacity-data-wrangling-mongo-db/blob/master/docs/ProjectReport.md

http://napitupulu-jon.appspot.com/posts/wrangling-openstreetmap.html

http://puwenning.github.io/2016/02/10/P3-project-openstreetmap-data-case-study/

http://fch808.github.io/Data%20Wrangling%20with%20MongoDB%20-%20Write-up.html